In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [29]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test['Transported'] = False
df = pd.concat([df_train, df_test])
df.drop(['Name', 'PassengerId'], axis=1, inplace=True)
df.head

<bound method NDFrame.head of      HomePlanet CryoSleep     Cabin    Destination   Age    VIP  RoomService  \
0        Europa     False     B/0/P    TRAPPIST-1e  39.0  False          0.0   
1         Earth     False     F/0/S    TRAPPIST-1e  24.0  False        109.0   
2        Europa     False     A/0/S    TRAPPIST-1e  58.0   True         43.0   
3        Europa     False     A/0/S    TRAPPIST-1e  33.0  False          0.0   
4         Earth     False     F/1/S    TRAPPIST-1e  16.0  False        303.0   
...         ...       ...       ...            ...   ...    ...          ...   
4272      Earth      True  G/1496/S    TRAPPIST-1e  34.0  False          0.0   
4273      Earth     False       NaN    TRAPPIST-1e  42.0  False          0.0   
4274       Mars      True   D/296/P    55 Cancri e   NaN  False          0.0   
4275     Europa     False   D/297/P            NaN   NaN  False          0.0   
4276      Earth      True  G/1498/S  PSO J318.5-22  43.0  False          0.0   

      Foo

In [30]:
df.shape[0] == df_train.shape[0] + df_test.shape[0]
df.head

<bound method NDFrame.head of      HomePlanet CryoSleep     Cabin    Destination   Age    VIP  RoomService  \
0        Europa     False     B/0/P    TRAPPIST-1e  39.0  False          0.0   
1         Earth     False     F/0/S    TRAPPIST-1e  24.0  False        109.0   
2        Europa     False     A/0/S    TRAPPIST-1e  58.0   True         43.0   
3        Europa     False     A/0/S    TRAPPIST-1e  33.0  False          0.0   
4         Earth     False     F/1/S    TRAPPIST-1e  16.0  False        303.0   
...         ...       ...       ...            ...   ...    ...          ...   
4272      Earth      True  G/1496/S    TRAPPIST-1e  34.0  False          0.0   
4273      Earth     False       NaN    TRAPPIST-1e  42.0  False          0.0   
4274       Mars      True   D/296/P    55 Cancri e   NaN  False          0.0   
4275     Europa     False   D/297/P            NaN   NaN  False          0.0   
4276      Earth      True  G/1498/S  PSO J318.5-22  43.0  False          0.0   

      Foo

In [31]:
df.isna().sum()

HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Transported       0
dtype: int64

In [32]:
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
df = df.drop(columns=['Cabin'])
df['Deck'] = df['Deck'].fillna('U')
df['Num'] = df['Num'].fillna(-1)
df['Side'] = df['Side'].fillna('U')

In [33]:
df['Deck'].value_counts()

Deck
F    4239
G    3781
E    1323
B    1141
C    1102
D     720
A     354
U     299
T      11
Name: count, dtype: int64

In [34]:
df['Deck'] = df['Deck'].map({'G' : 0, 'F' : 1, 'E' : 2, 'D' : 3, 'C' : 4, 'B' : 5, 'A' : 6, 'U' : 7, 'T' : 8})
df['Side'] = df['Side'].map({'U' : 0, 'P' : 1, 'S' : 2})

In [35]:
impute_lis = ['Age', 'VIP', "Num", "CryoSleep", 'Side', 'Deck', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
rest = list(set(df.columns) - set(impute_lis))
df_rest = df[rest]
imp = KNNImputer(n_neighbors=5)
df_imputed = imp.fit_transform(df[impute_lis])
df_imputed = pd.DataFrame(df_imputed, columns=impute_lis)
df = pd.concat([df_rest.reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)

In [36]:
df['HomePlanet'] = df['HomePlanet'].fillna('U')
df['Destination'] = df['Destination'].fillna('U')
category_colls = ['HomePlanet', 'Destination']

for col in category_colls:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)

    '''df = pd.concat'''


In [37]:
df.drop(columns=category_colls)


,Transported,Age,VIP,Num,CryoSleep,Side,Deck,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_U,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_U
0,False,39.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False,False,False,True,False
1,True,24.0,0.0,0.0,0.0,2.0,1.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,False,True,False
2,False,58.0,1.0,0.0,0.0,2.0,6.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,False,True,False
3,False,33.0,0.0,0.0,0.0,2.0,6.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,False,True,False
4,True,16.0,0.0,1.0,0.0,2.0,1.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,False,34.0,0.0,1496.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,False,False,False,False,True,False
12966,False,42.0,0.0,-1.0,0.0,0.0,7.0,0.0,847.0,17.0,10.0,144.0,True,False,False,False,False,False,True,False
12967,False,24.6,0.0,296.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,True,False,False,False
12968,False,30.2,0.0,297.0,0.0,1.0,3.0,0.0,2680.0,0.0,0.0,523.0,False,True,False,False,False,False,False,True


In [38]:
df.head()

,Destination,Transported,HomePlanet,Age,VIP,Num,CryoSleep,Side,Deck,RoomService,...,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_U,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_U
0,TRAPPIST-1e,False,Europa,39.0,0.0,0.0,0.0,1.0,5.0,0.0,...,0.0,0.0,False,True,False,False,False,False,True,False
1,TRAPPIST-1e,True,Earth,24.0,0.0,0.0,0.0,2.0,1.0,109.0,...,549.0,44.0,True,False,False,False,False,False,True,False
2,TRAPPIST-1e,False,Europa,58.0,1.0,0.0,0.0,2.0,6.0,43.0,...,6715.0,49.0,False,True,False,False,False,False,True,False
3,TRAPPIST-1e,False,Europa,33.0,0.0,0.0,0.0,2.0,6.0,0.0,...,3329.0,193.0,False,True,False,False,False,False,True,False
4,TRAPPIST-1e,True,Earth,16.0,0.0,1.0,0.0,2.0,1.0,303.0,...,565.0,2.0,True,False,False,False,False,False,True,False


In [39]:
bill_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df['amt_spent'] = df[bill_cols].sum(axis=1)
df['std_amt_spent'] = df[bill_cols].std(axis=1)
df['mean_amt_spent'] = df[bill_cols].mean(axis=1)

In [40]:
df.corr()

ValueError: could not convert string to float: 'TRAPPIST-1e'